# 1. Import dependencies

In [2]:
pip install stable-baselines3[extra]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygame

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyglet

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os  # operating systme library to save and load model
import gym  # openAI gym simulated environments- building the environment and working with ore-existing
from stable_baselines3 import PPO  # one of the algorithms of stable baselines
from stable_baselines3.common.vec_env import DummyVecEnv  # instead of vectorizing the environements for ML
from stable_baselines3.common.evaluation import evaluate_policy  # makes it easier to test out if the model is performing

# 2. Load Environment

In [2]:
environment_name = "CartPole-v1"
env = gym.make(environment_name)

In [3]:
# understanding the environment - writing a loop to test that environment
episodes = 5  # testing the environment 5 times 
for episode in range(1, episodes+1):
    state = env.reset()  # reseting environment by getting an array of 4 initial set of observations of that perticular environment
    done = False
    score = 0 
    
    while not done:  
        env.render()  # to view the graphical representation of environment
        action = env.action_space.sample()  # generating a random action, 0 or 1 - here action_space is Discrete(2) -> {0, 1}
        n_state, reward, done, info = env.step(action)  # passing random action to environment (passing 0 or 1)
        score+=reward  # accumulating rewards
    print('Episode:{} Score:{}'.format(episode, score))
    print(n_state)  # [cart position, cart velocity, pole angle, pole angular velocity]

2022-07-11 14:42:23.463 Python[2435:72796] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/21/_7zvs2610fxgclkhl47pkcm40000gn/T/org.python.python.savedState


Episode:1 Score:9.0
[-0.09145254 -1.0301107   0.21023993  1.7724929 ]
Episode:2 Score:30.0
[ 0.24633534  1.5257928  -0.2392042  -2.1940482 ]
Episode:3 Score:34.0
[ 0.17071104  1.2201657  -0.23982005 -2.1085558 ]
Episode:4 Score:15.0
[ 0.1510234  1.3251384 -0.245061  -2.2469547]
Episode:5 Score:12.0
[ 0.12424244  1.5559841  -0.21802317 -2.4916277 ]


In [6]:
env.close()

# Understanding The Environment

In [11]:
# 0-push cart to left, 1-push cart to the right --> discrete(2)
env.action_space.sample()

1

In [10]:
# [cart position, cart velocity, pole angle, pole angular velocity] --> Box(4) 
env.observation_space.sample()

array([ 2.5595155e+00,  9.3737701e+37, -3.7165624e-01, -2.8364767e+38],
      dtype=float32)

# 3 Training

RL Algorithms:
    1. Model-Based - predict future states to generate best possible action
        1. Learn the Model
        2. Given the Model --> AlphaZero
    2. Model-Free ( a lot of developments are happening) - uses current state to make predictions
        1.Policiy Optimization --> PPO (Proximal Policy Optimization ) / A2C / DQN
        2.Q-Learning
using best algorithms based on the action space in Stable Baseline:
    A2C - works on all types of action space (Box, Discrete, MultiDistricrete, MultiBinary, MultiProcessing)
    PPO - works on all types of action space (Box, Discrete, MultiDistricrete, MultiBinary, MultiProcessing)
    DQN - only on Discrete

In [4]:
log_path = os.path.join('Training', 'Logs')  # define a log_path where we can monitor the performance of the model

In [5]:
env = gym.make(environment_name)  
env = DummyVecEnv([lambda: env])  # wrapping the environment inside the DummyVecEnv with lambda environment creation function
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)  # defining the model - defining the policiy (multulayer perceptron policy (standard NN))

Using cpu device


In [6]:
model.learn(total_timesteps=20000)  # traning the model

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 81   |
|    iterations      | 1    |
|    time_elapsed    | 25   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 153         |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009250168 |
|    clip_fraction        | 0.0952      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000464    |
|    learning_rate        | 0.0003      |
|    l

In [18]:
PPO??  # cheking the parameters of PPO

# 4. Save and Reload Model

In [70]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_Cartpole')

In [71]:
PPO_path

'Training/Saved Models/PPO_model_Cartpole'

In [74]:
model.save(PPO_path)

In [68]:
del model

In [69]:
model = PPO.load(PPO_path, env)  # reloading does not work

FileNotFoundError: [Errno 2] No such file or directory: 'PPO_model.zip'

In [75]:
model.learn(total_timesteps=1000)

Logging to Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 2775 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


# 4. Evaluation -- > PPO model is considered solve ->  score or average reward > 200

In [76]:
from stable_baselines3.common.evaluation import evaluate_policy  # testing how well the model is performing

In [94]:
evaluate_policy(model, env, n_eval_episodes=10, render=True) # average reward (if above 200 then is considered solved, standard deviation

(500.0, 0.0)

In [95]:
env.close()

# 5. Test Model --> testing it with the model interacting with the environment and not random actions

In [100]:
episodes = 5  # testing the environment 5 times 
for episode in range(1, episodes+1):
    obs = env.reset()  # current observations
    done = False
    score = 0 
    
    while not done:  
        env.render()  # to view the graphical representation of environment
        action, _ = model.predict(obs)  # using the model instead to generate the best action depending on the current observation of random actions!
        obs, reward, done, info = env.step(action)  # passing random action to environment (passing 0 or 1)
        score+=reward  # accumulating rewards
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


In [101]:
env.close()

In [103]:
obs = env.reset()  # current observation

In [104]:
model.predict(obs)  # passing the current observations to the model --> getting the best possible action through the model based on the observation of the current environment

(array([1]), None)

In [105]:
action, _ = model.predict(obs)

In [106]:
env.step(action)  # state after taking action, reward(1 for every step taken), done, info

(array([[-0.00815107,  0.17906141,  0.00569474, -0.26315492]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

# 6. Viewing Logs in Tensorboard (useful for big models)

In [112]:
pip install tensorboard

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [128]:
training_log_path = os.path.join(log_path, 'PPO_9')

In [129]:
!tensorboard --logdir={training_log_path}

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# 7. Adding a callback to the training Stage (useful for big models)

In [119]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [120]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [121]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [136]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)  # stopping the training at 200 score
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000,  # on every 10000 steps it checks if it has passed score 200, and if so it saves the best new model
                             best_model_save_path=save_path, 
                             verbose=1)

In [137]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [138]:
model.learn(total_timesteps=20000, callback=eval_callback)  # 

Logging to Training/Logs/PPO_11
-----------------------------
| time/              |      |
|    fps             | 2616 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1803        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008486377 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0325     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.81        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0146     |
|    value_loss           | 52.5        |
-----------------------------------------
--

In [133]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [134]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(361.6, 99.7579069547873)

In [139]:
env.close()

# 8. Changing Policies - using a different network architecture

In [140]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]  # new neural architecture defined

In [141]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [142]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 2046 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1171         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0140363565 |
|    clip_fraction        | 0.164        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.683       |
|    explained_variance   | -0.00859     |
|    learning_rate        | 0.0003       |
|    loss                 | 3.31         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.021       |
|    value_loss           | 19.3         |
------------------------------------------
----------------

# 9. Using an Alternate Algorithm - DQN instead of PPO

In [143]:
from stable_baselines3 import DQN

In [144]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [145]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.96     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4316     |
|    time_elapsed     | 0        |
|    total_timesteps  | 85       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.924    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 5098     |
|    time_elapsed     | 0        |
|    total_timesteps  | 159      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.875    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6178     |
|    time_elapsed     | 0        |
|    total_timesteps  | 263      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 8119     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2564     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 8146     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2649     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 8121     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2704     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 8374     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4865     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 8385     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4932     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 8389     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5004     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 8255     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7294     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 8274     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7407     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 8272     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7511     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 8299     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9353     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 8312     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9453     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 8314     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9563     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 8325     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11814    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 8338     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11919    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 8328     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11988    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 8392     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14364    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 8395     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14453    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 8399     |
|    time_elapsed     | 1        |
|    total_timesteps  | 14566    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 8455     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16797    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 8451     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16901    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 8448     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16989    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 8471     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19137    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 8477     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19240    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 8485     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19367    |
----------------------------------
----------------------------------
| rollout/          

In [146]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [147]:
model.save(dqn_path)

In [148]:
model = DQN.load(dqn_path, env=env)

In [149]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.4, 0.8)

In [150]:
env.close()